In [ ]:
import cv2
from ultralytics import YOLO
import pandas as pd
import time
from datetime import datetime
import numpy as np
import requests
import json
import os

# Configuration
ESP8266_IP = "192.168.42.42"  
SEND_DELAY = 0.1  
CSV_FILE = 'count_data_realtime.csv'

class ESPConnection:
    def __init__(self, ip_address):
        self.ip = ip_address
        self.last_sent_time = 0
        self.last_sent_count = None
        
    def send_count(self, count):
        current_time = time.time()
        
        if (count != self.last_sent_count and 
            current_time - self.last_sent_time >= SEND_DELAY):
            
            try:
                url = f'http://{self.ip}/update-cv-count'
                response = requests.post(url, 
                                      json={'count': count},
                                      headers={'Content-Type': 'application/json'},
                                      timeout=1)
                
                if response.status_code == 200:
                    self.last_sent_count = count
                    self.last_sent_time = current_time
                    print(f"Successfully sent count {count} to ESP8266")
                else:
                    print(f"Failed to send count. Status code: {response.status_code}")
                    
            except requests.exceptions.RequestException as e:
                print(f"Error sending count to ESP8266: {e}")

def create_count_display(df):
    """Create a visual display of the count data"""
    height = 400
    width = 300
    display = np.ones((height, width, 3), dtype=np.uint8) * 255
    
    cv2.putText(display, "Count by Minute", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

    cv2.putText(display, "Time", (10, 70), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    cv2.putText(display, "Count", (200, 70), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

    cv2.line(display, (10, 80), (width-10, 80), (0, 0, 0), 1)
    
    last_entries = df.tail(10)
    for i, (_, row) in enumerate(last_entries.iterrows()):
        y_pos = 110 + i * 30
        time_str = row['Time'].strftime('%H:%M')
        cv2.putText(display, time_str, (10, y_pos), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
        cv2.putText(display, str(row['Count']), (200, y_pos), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    
    return display

def append_to_csv(data, filename):
    """Append a single row of data to CSV file"""
    # If file doesn't exist, create it with headers
    if not os.path.exists(filename):
        data.to_csv(filename, index=False)
    else:
        # Append without headers
        data.to_csv(filename, mode='a', header=False, index=False)

def main():
    # Initialize ESP connection
    esp = ESPConnection(ESP8266_IP)
    
    # Initialize YOLO model
    model = YOLO("yolov9c.pt")
    
    # Initialize video capture and writer
    cap = cv2.VideoCapture(1)
    assert cap.isOpened(), "Error opening webcam"
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    video_writer = cv2.VideoWriter("res//output_video.mp4", 
                                 cv2.VideoWriter_fourcc(*"mp4v"), 
                                 30, 
                                 (frame_width, frame_height))
    
    # Initialize tracking variables
    count_line_y = frame_height // 2
    count = 0
    tracked_objects = {}
    
    # Initialize time and DataFrame for count tracking
    start_time = time.time()
    current_minute = -1
    
    # Load existing CSV data if available
    if os.path.exists(CSV_FILE):
        df = pd.read_csv(CSV_FILE)
        df['Time'] = pd.to_datetime(df['Time'])  # Convert Time column back to datetime
    else:
        df = pd.DataFrame(columns=['Time', 'Count'])
    
    while True:
        success, frame = cap.read()
        if not success:
            print("Failed to grab frame")
            break
            
        # Update count data every minute
        elapsed_time = time.time() - start_time
        new_minute = int(elapsed_time // 60)
        
        if new_minute > current_minute:
            new_row = pd.DataFrame({
                'Time': [datetime.now().replace(second=0, microsecond=0)],
                'Count': [count]
            })
            # Append new row to CSV
            append_to_csv(new_row, CSV_FILE)
            # Update DataFrame for display
            df = pd.concat([df, new_row], ignore_index=True)
            current_minute = new_minute
            
        # Run YOLO detection and tracking
        results = model.track(frame, persist=True, tracker='botsort.yaml', iou=0.2)
        
        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
            track_ids = results[0].boxes.id.cpu().numpy().astype(int)
            class_ids = results[0].boxes.cls.cpu().numpy().astype(int)
            
            for box, track_id, class_id in zip(boxes, track_ids, class_ids):
                if class_id == 0:  # Person class
                    x1, y1, x2, y2 = box
                    center_y = (y1 + y2) // 2
                    
                    if track_id in tracked_objects:
                        prev_center_y = tracked_objects[track_id]
                        
                        # Update count and send to ESP8266
                        if prev_center_y < count_line_y and center_y >= count_line_y:
                            count += 1
                            esp.send_count(count)
                        elif prev_center_y > count_line_y and center_y <= count_line_y:
                            count = max(0, count - 1)
                            esp.send_count(count)
                    
                    tracked_objects[track_id] = center_y
                    
                    # Draw bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # Draw counting line and display count
        cv2.line(frame, (0, count_line_y), (frame_width, count_line_y), 
                (255, 0, 0), 2)
        cv2.putText(frame, f"Count: {count}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(frame, f"Time: {int(elapsed_time)}s", (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        # Create and display count data window
        count_display = create_count_display(df)
        cv2.imshow('Count Data', count_display)
        
        # Display and record
        video_writer.write(frame)
        cv2.imshow('Processed Feed', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Save final count before closing
    final_row = pd.DataFrame({
        'Time': [datetime.now().replace(second=0, microsecond=0)],
        'Count': [count]
    })
    append_to_csv(final_row, CSV_FILE)
    
    # Cleanup
    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


0: 384x640 3 persons, 1 chair, 297.8ms
Speed: 3.9ms preprocess, 297.8ms inference, 6.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 chair, 274.5ms
Speed: 1.4ms preprocess, 274.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 1 chair, 249.7ms
Speed: 1.2ms preprocess, 249.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 1 chair, 217.2ms
Speed: 1.2ms preprocess, 217.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)



2025-01-04 18:05:56.035 python[4833:103723] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-04 18:05:56.035 python[4833:103723] +[IMKInputSession subclass]: chose IMKInputSession_Modern


0: 384x640 5 persons, 1 chair, 221.6ms
Speed: 1.2ms preprocess, 221.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 1 chair, 215.8ms
Speed: 1.2ms preprocess, 215.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 1 bowl, 217.2ms
Speed: 1.1ms preprocess, 217.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 221.2ms
Speed: 1.9ms preprocess, 221.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 219.2ms
Speed: 1.5ms preprocess, 219.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 218.0ms
Speed: 1.3ms preprocess, 218.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 225.4ms
Speed: 1.2ms preprocess, 225.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 239.9ms
Speed: 1.7ms preprocess, 239.9ms inferenc